In [61]:
import numpy as np
import torch
import torch.nn as nn

In [62]:
class LongShortTermMemoryModel(nn.Module):

    def __init__(self, encoding_size):
        super(LongShortTermMemoryModel, self).__init__()

        self.lstm = nn.LSTM(encoding_size, 128)  # 128 is the state size
        self.dense = nn.Linear(128, encoding_size)  # 128 is the state size

    def reset(self):  # Reset states prior to new input sequence
        zero_state = torch.zeros(1, 1, 128)  # Shape: (number of layers, batch size, state size)
        self.hidden_state = zero_state
        self.cell_state = zero_state

    def logits(self, x):  # x shape: (sequence length, batch size, encoding size)
        out, (self.hidden_state, self.cell_state) = self.lstm(x, (self.hidden_state, self.cell_state))
        return self.dense(out.reshape(-1, 128))

    def f(self, x):  # x shape: (sequence length, batch size, encoding size)
        return torch.softmax(self.logits(x), dim=1)

    def loss(self, x, y):  # x shape: (sequence length, batch size, encoding size), y shape: (sequence length, encoding size)
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

In [1]:
##Many to one
index_to_char = [' ', 'h', 'a', 't', 'r', 'c', 'b', 'n', 'k', 'o', 'l', 'v', 'p', 'u']
char_encodings = np.identity(len(index_to_char), dtype=float).tolist()

emoji = {
    "hat": ["\U0001F3A9", "hat "],
    "rat":["\U0001F401", "rat "],
    "cat":["\U0001F408", "cat "],
    "bank":["\U0001F3E6", "bank"],
    "olav":["\U0001F468", "olav"],
    "cap":["\U0001F9E2", "cap "],
    "knut" :["\U0001F349", "knut"]
}


def emoji_index(y: int):
    i = 0
    for value in emoji.values():
        if i == y:
            return value[0]
        i += 1

#print(emojies)
emojies = np.eye(len(emoji))
encoding_size = len(char_encodings)



letter_dict = {index_to_char[i]: i for i in range(0, len(index_to_char))}

def letter(x: str):
    char = char_encodings[letter_dict[x]]
    return char

print(len(index_to_char))
x_train = torch.tensor([
                       [[letter('h')], [letter('a')], [letter('t')],[letter(' ')]],
                       [[letter('r')], [letter('a')], [letter('t')],[letter(' ')]],
                       [[letter('c')], [letter('a')], [letter('t')],[letter(' ')]],
                       [[letter('b')], [letter('a')], [letter('n')],[letter('k')]],
                       [[letter('o')], [letter('l')], [letter('a')],[letter('v')]],
                       [[letter('c')], [letter('a')], [letter('p')],[letter(' ')]],
                       [[letter('k')], [letter('n')], [letter('u')],[letter('t')]],
                        ], dtype=torch.float)
y_train = torch.tensor([
                        [emojies[0], emojies[0], emojies[0], emojies[0]],
                        [emojies[1], emojies[1], emojies[1], emojies[1]],
                        [emojies[2], emojies[2], emojies[2], emojies[2]],
                        [emojies[3], emojies[3], emojies[3], emojies[3]],
                        [emojies[4], emojies[4], emojies[4], emojies[4]],
                        [emojies[5], emojies[5], emojies[5], emojies[5]],
                        [emojies[6], emojies[6], emojies[6], emojies[6]]
                       ], dtype=torch.float)


model = LongShortTermMemoryModel(encoding_size)

NameError: name 'np' is not defined

In [113]:
optimizer = torch.optim.RMSprop(model.parameters(), 0.001)
for epoch in range(500):
    for i in range(x_train.size()[0]):#Loops for each letter to emoji
        model.reset()
        model.loss(x_train[i], y_train[i]).backward()
        optimizer.step()
        optimizer.zero_grad()


In [118]:
def get_emoji(emo: str):
    y = -1
    model.reset()
    for i in range(len(emo)):
        char_i = index_to_char.index(emo[i])
        y = model.f(torch.tensor([[char_encodings[char_i]]]))
    print(emoji_index(y.argmax(1)))

get_emoji("knut")


🍉
